In [2]:
import os
import dash
from dash import dash_table, html, dcc
import pandas as pd
import requests
from dash.dependencies import Input, Output

# Initialize the Dash app
app = dash.Dash(__name__)

def fetch_all_drivers():
    try:
        # fetch drivers only for the 2024 season
        response = requests.get("http://ergast.com/api/f1/2024/drivers.json")
        if response.status_code == 200:
            drivers = response.json()['MRData']['DriverTable']['Drivers']
            return [{'label': f"{driver['givenName']} {driver['familyName']}", 'value': driver['driverId']} for driver in drivers]
    except Exception as e:
        # Log the error (locally) if needed
        print(f"Error fetching drivers: {e}")
    return []

def fetch_lap_positions(driver_id):
    all_races_data = []
    base_url = "http://ergast.com/api/f1"
    
    try:
        # fetch regular and sprint race data for the 2024 season
        season = 2024
        response = requests.get(f"{base_url}/{season}/drivers/{driver_id}/results.json")
        if response.status_code != 200:
            return None  # gracefully return None to indicate error
        
        races = response.json()['MRData']['RaceTable']['Races']
        
        for race in races:
            race_name = race['raceName']
            round_number = race['round']
            
            # fetch the grid position from the race results
            start_position = int(race['Results'][0]['grid'])
            
            # fetch lap times for the first lap in the main race
            lap_times_response = requests.get(f"{base_url}/{season}/{round_number}/laps/1.json")
            if lap_times_response.status_code != 200:
                continue
            
            lap_times = lap_times_response.json()['MRData']['RaceTable']['Races'][0].get('Laps', [])
            
            if lap_times:
                lap_1_position = next((int(timing['position']) for timing in lap_times[0]['Timings'] if timing['driverId'] == driver_id), None)
                
                diff_start_lap1 = (start_position - lap_1_position) if lap_1_position else None
                
                all_races_data.append([season, race_name, "Race", start_position, lap_1_position, diff_start_lap1])
            
            # fetch sprint race data
            sprint_response = requests.get(f"{base_url}/{season}/{round_number}/sprint.json")
            if sprint_response.status_code == 200 and sprint_response.json()['MRData']['RaceTable']['Races']:
                sprint_race = sprint_response.json()['MRData']['RaceTable']['Races'][0]
                sprint_name = sprint_race['raceName']
                sprint_start_position = int(sprint_race['SprintResults'][0]['grid'])
                sprint_finish_position = int(sprint_race['SprintResults'][0]['position'])
                diff_sprint = sprint_start_position - sprint_finish_position
                
                all_races_data.append([season, sprint_name, "Sprint", sprint_start_position, sprint_finish_position, diff_sprint])
    
    except Exception as e:
        # Log the error (locally) if needed
        print(f"Error fetching lap positions: {e}")
        return None
    
    return all_races_data

def calculate_stats(df):
    total_diff = df['Difference'].sum()
    places_lost = len(df[df['Difference'] < 0])
    places_gained = len(df[df['Difference'] > 0])
    no_change = len(df[df['Difference'] == 0])
    total_races = len(df)
    
    return {
        'Total places lost/gained after lap 1': total_diff,
        'Total races with places lost after lap 1': f"{places_lost} ({places_lost/total_races:.1%})",
        'Total races with places gained after lap 1': f"{places_gained} ({places_gained/total_races:.1%})",
        'Total races with no change': f"{no_change} ({no_change/total_races:.1%})"
    }

# Define the layout of the Dash app
app.layout = html.Div([
    html.H1("F1 Driver Performance Dashboard", style={'textAlign': 'center'}),

    html.Div([
        html.H3("Purpose of Analysis"),
        html.P("This analysis was inspired by a Reddit post questioning whether Lando Norris tends to lose out on his race starts. "
               "You can find the original post here: "),
        dcc.Link("Does Lando Norris really lose out on his race starts? (Reddit)", href="https://www.reddit.com/r/formula1/comments/1egodxm/does_lando_norris_really_lose_out_on_his_race/", target="_blank"),
    ], style={'padding': '20px'}),

    html.Div([
        html.H3("API Information"),
        html.P("The data for this dashboard is fetched using the Ergast API, which provides F1 race data, including race results, "
               "lap times, and more. Due to API constraints, fetching data might take a little time.")
    ], style={'padding': '20px', 'backgroundColor': '#f9f9f9'}),

    dcc.Dropdown(
        id='driver-dropdown',
        options=fetch_all_drivers(),
        value='norris',
        clearable=False,
        style={'width': '50%', 'margin': 'auto'}
    ),

    html.Div(id='loading-message', children="Please wait, the data might take a moment to load due to API constraints.", style={'color': 'red', 'textAlign': 'center', 'marginTop': '20px'}),
    
    html.Div([
        html.Div(id='stats-output', style={'padding': '20px'}),
        html.Div(id='driver-stats-header', style={'textAlign': 'center', 'marginBottom': '20px'}),
    ], style={'marginTop': '20px'}),

    # Now the table occupies the entire width, placed below the stats
    html.Div([
        dash_table.DataTable(
            id='performance-table',
            columns=[{'name': col, 'id': col} for col in ["Season", "Race Name", "Event Type", "Start Position", "Position After Lap 1", "Difference"]],
            style_table={'overflowX': 'auto', 'width': '100%'},
            style_cell={'textAlign': 'center'},
            style_data_conditional=[
                {
                    'if': {'column_id': 'Difference', 'filter_query': '{Difference} > 0'},
                    'backgroundColor': 'green',
                    'color': 'white'
                },
                {
                    'if': {'column_id': 'Difference', 'filter_query': '{Difference} < 0'},
                    'backgroundColor': 'red',
                    'color': 'white'
                },
                {
                    'if': {'column_id': 'Start Position', 'filter_query': '{Start Position} = 1'},
                    'backgroundColor': 'gold',
                    'color': 'black'
                }
            ]
        )
    ], style={'width': '100%', 'marginTop': '20px'}),

    html.Div([
        html.H3("Contact for Feedback"),
        html.P("For any feedback, please feel free to contact me at: pamperedrebel@gmail.com. "
               "I am still actively working on this project.", style={'textAlign': 'center'})
    ], style={'padding': '20px', 'backgroundColor': '#f1f1f1', 'marginTop': '40px'})
])

@app.callback(
    [Output('performance-table', 'data'),
     Output('stats-output', 'children'),
     Output('driver-stats-header', 'children'),
     Output('loading-message', 'style')],
    [Input('driver-dropdown', 'value')]
)
def update_table(selected_driver):
    races_data = fetch_lap_positions(selected_driver)
    
    if races_data is None:
        # Show alert message if the data doesn't load
        return [], [], "", {'color': 'red', 'textAlign': 'center', 'marginTop': '20px', 'display': 'block'}
    
    df = pd.DataFrame(races_data, columns=["Season", "Race Name", "Event Type", "Start Position", "Position After Lap 1", "Difference"])
    
    # calculate stats for 2024 only
    stats = calculate_stats(df[df['Season'] == 2024])
    stats_html = [html.H3("Statistics for 2024")] + [html.P(f"{k}: {v}") for k, v in stats.items()]
    
    driver_name = next(d['label'] for d in fetch_all_drivers() if d['value'] == selected_driver)
    
    return df.to_dict('records'), stats_html, f"Performance Stats for {driver_name}", {'display': 'none'}


if __name__ == "__main__":
    app.run(debug=True)